In [ ]:
## IMPORT NECESSARY LIBRARIES 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import autograd.numpy as npg
from autograd import grad, elementwise_grad as e_grad
import time
from datetime import datetime,date
import math
import requests
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot, layout
from bokeh.transform import cumsum
import bankroll
import glob
import os
import time

In [ ]:
class AdamOptim():
    def __init__(self, eta=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.m_dw, self.v_dw = 0, 0
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.eta = eta
    def update(self, t, w, dw):
        ## dw, db are from current minibatch
        ## momentum beta 1
        # *** weights *** #
        self.m_dw = self.beta1*self.m_dw + (1-self.beta1)*dw

        ## rms beta 2
        # *** weights *** #
        self.v_dw = self.beta2*self.v_dw + (1-self.beta2)*(dw**2)

        ## bias correction
        m_dw_corr = self.m_dw/(1-(self.beta1**t)+self.epsilon)
        v_dw_corr = self.v_dw/(1-(self.beta2**t)+self.epsilon)

        ## update weights and biases
        w = w - self.eta*(m_dw_corr/(np.sqrt(v_dw_corr)+self.epsilon))
        return w
    def update_v(self, t, w, dw):
        ## dw, db are from current minibatch
        ## momentum beta 1
        # *** weights *** #
        self.m_dw = np.add(np.multiply(self.beta1,self.m_dw),np.multiply((1-self.beta1), dw))

        ## rms beta 2
        # *** weights *** #
        self.v_dw = np.add(np.multiply(self.beta2, self.v_dw), np.multiply((1-self.beta2),np.square(dw)))

        ## bias correction
        m_dw_corr = np.divide(self.m_dw,np.subtract((1+self.epsilon),np.power(self.beta1,t)))
        v_dw_corr = np.divide(self.v_dw,np.subtract((1+self.epsilon),np.power(self.beta2,t)))

        ## update weights and biases
        w = np.subtract(w, np.multiply(self.eta, np.divide(m_dw_corr, np.add(np.sqrt(v_dw_corr), self.epsilon))))
        return w

In [ ]:
def norm(x):
    x = x - np.min(x)
    x = x / np.sum(x)
    return x

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def tanh_refined(x):
    return (np.tanh(x)+1.0)/2.0

def get_weights(wts):
#     wts = sigmoid(wts)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts)
    return wts


In [ ]:

def get_weights_v(wts):
#     wts = sigmoid(wts)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts, axis=1, keepdims=True)
    return wts

def get_return_v(wts):
    wts = get_weights_v(wts)
    port_ret = npg.sum(log_ret_mean * wts, axis=1)
    port_ret = npg.power((port_ret + 1), 252) - 1
    return port_ret
    
def get_risk_v(wts):
    wts = get_weights_v(wts)
    port_sd = npg.sqrt(npg.sum(wts * npg.dot(wts, cov_mat.T), axis=1))
    return port_sd

def get_sharpe_v(wts):
    port_ret = get_return_v(wts)
    port_sd = get_risk_v(wts)
    sr = port_ret / port_sd
    return sr

def get_loss_v(risk_target=None, return_target=None):
    weight = 1.0e3
    if risk_target is not None and return_target is not None:
        return lambda x: ((npg.square(risk_target - get_risk_v(x)))+
                          (npg.square(return_target - get_return_v(x))))
    elif risk_target is not None:
        return lambda x: ((weight*npg.square(risk_target - get_risk_v(x))) - get_return_v(x))/(weight + 1.0)
    elif return_target is not None:
        return lambda x: ((weight*npg.square(return_target - get_return_v(x))) - get_sharpe_v(x))/(weight + 1.0)
    return lambda x: npg.negative(get_sharpe_v(x))

In [ ]:
wts = np.random.random((5,10))

In [ ]:
loss = get_loss_v()
loss(wts)

In [ ]:
def filterTickers(ticks, tick_allowed):
    return [tick for tick, allowed in zip(ticks, tick_allowed) if allowed]

def get_return(wts):
    wts = npg.array(filterTickers(wts, tick_allowed))
    wts = get_weights(wts)
    port_ret = npg.sum(log_ret_mean * wts)
    port_ret = (port_ret + 1) ** 252 - 1
    return port_ret
    
def get_risk(wts):
#     print(wts)
    wts = npg.array(filterTickers(wts, tick_allowed))
#     print(wts)
    wts = get_weights(wts)
    port_sd = npg.sqrt(npg.dot(wts.T, npg.dot(cov_mat, wts)))
    return port_sd

def get_sharpe(wts):
    port_ret = get_return(wts)
    port_sd = get_risk(wts)
    sr = port_ret / port_sd
    return sr

def get_loss(risk_target=None, return_target=None):
    weight = 1.0e3
    if risk_target is not None and return_target is not None:
        return lambda x: ((npg.square(risk_target - get_risk(x)))+
                          (npg.square(return_target - get_return(x))))
    elif risk_target is not None:
        return lambda x: ((weight*npg.square(risk_target - get_risk(x))) - get_return(x))/(weight + 1.0)
    elif return_target is not None:
        return lambda x: ((weight*npg.square(return_target - get_return(x))) - get_sharpe(x))/(weight + 1.0)

In [ ]:
def get_gd_weights(weights_size, loss_fun, batch, iterations, LR, scorings, initial=None):
    # Optimize weights using gradient descent.
    if initial is not None:
        best_weights = get_weights(initial) + np.random.uniform(size=weights_size)
    else:
        best_weights = np.random.uniform(size=weights_size)
    training_gradient_fun = elementwise_grad(loss_fun)
    scores = np.zeros((len(scorings), batch, iterations))
    for b in range(batch):
        if initial is not None:
            wts = get_weights(initial) + np.random.uniform(size=weights_size)
        else:
            wts = np.random.uniform(size=weights_size)
#         wts = np.random.uniform(size=weights_size)
        for i in range(iterations):
            wts = wts - training_gradient_fun(wts) * LR
            for s in range(len(scorings)):
                print(scores[s, b, i].shape)
                print(scorings[s](wts).shape)
                scores[s, b, i] = scorings[s](wts)
        if loss_fun(wts) < loss_fun(best_weights):
            best_weights = wts
    return best_weights, scores

def get_gd_weights_adam(weights_size, loss_fun, batch, iterations, LR, scorings, initial=None):
    adam = AdamOptim(eta=LR)
    # Optimize weights using gradient descent.
    if initial is not None:
        best_weights = get_weights(initial) + np.random.uniform(size=weights_size)
    else:
        best_weights = np.random.uniform(size=weights_size)
    training_gradient_fun = grad(loss_fun)
    scores = np.zeros((len(scorings), batch, iterations))
    for b in range(batch):
        if initial is not None:
            wts = get_weights(initial) + np.random.uniform(size=weights_size)
        else:
            wts = np.random.uniform(size=weights_size)*4
#         wts = np.random.uniform(size=weights_size)
        for i in range(iterations):
            dw = training_gradient_fun(wts)
            wts = adam.update(i+1,wts, dw)
            for s in range(len(scorings)):
#                 print(scorings[s](wts))
                scores[s, b, i] = scorings[s](wts)
        if loss_fun(wts) < loss_fun(best_weights):
            best_weights = wts
    return best_weights, scores



def get_gd_weights_adam_batched(weights_size, loss_fun_v, batch, iterations, LR, scorings, initial=None):
    adam = AdamOptim(eta=LR)
    training_gradient_fun_v = e_grad(loss_fun_v)
    scores = np.zeros((len(scorings), batch, iterations))
    wts = np.random.uniform(size=(batch, weights_size))*4
    print(wts.shape)
    for i in range(iterations):
        dw = training_gradient_fun_v(wts)
        wts = adam.update(i+1,wts, dw)
        for s in range(len(scorings)):
            scores[s, :, i] = scorings[s](wts)
    best_weights = wts[np.argmin(loss_fun_v(wts)),:]
    return best_weights, scores

def get_gd_weights_adam_batched_exp(weights_size, loss_fun_v, batch, iterations, LR, scorings, initial=None):
    adam = AdamOptim(eta=LR)
    training_gradient_fun_v = e_grad(loss_fun_v)
    scores = np.zeros((len(scorings), batch, iterations))
    wts = np.random.uniform(size=(batch, weights_size))*4
    print(wts.shape)
    for i in range(iterations):
        dw = training_gradient_fun_v(wts)
        wts = adam.update_v(i+1,wts, dw)
        for s in range(len(scorings)):
            scores[s, :, i] = scorings[s](wts)
    best_weights = wts[np.argmin(loss_fun_v(wts)),:]
    return best_weights, scores

## Set constants

In [ ]:
num_profiles = 5000
inflation_rate = 0.06

start_date = '01/06/2000'
end_date = '01/06/2021'

plt.rnparams['figure.figsize'] = [15, 5]
np.random.seed(42)

### Define Ticks

In [ ]:
ticks = [
    "FITLX",
    "FNIDX",
    "FNDSX",
    "SUSA",
    "IQSU", 
    "USSG",
    "SUSB",
    "SNPE",
    "SUSL",
    "EAGG",
    "WSBFX",
    "NEXTX",
    "CASH",
]
tick_names = [
    "Fidelity® U.S. Sustainability Index Fund",
    "Fidelity® International Sustainability Index Fund",
    "Fidelity® Sustainability Bond Index Fund",
    "iShares MSCI USA ESG Select ETF", 
    "IQ Candriam ESG US Equity ETF", 
    "Xtrackers MSCI USA ESG Leaders Eq ETF",
    "iShares ESG 1-5 Year USD Corp Bd ETF",
    "Xtrackers S&P 500 ESG ETF",
    "iShares ESG MSCI USA Leaders ETF",
    "iShares ESG Aware U.S. Aggregate Bond ETF",
    "Boston Trust Walden Balanced Fund", 
    "Shelton Green Alpha Fund",
    "Cash",
]
tick_allowed = [
    True,
    True,
    True,
    True,
    True, 
    True,
    True,
    True,
    True,
    True,
    False,
    False,
    False,
]
ticks_filtered = filterTickers(ticks, tick_allowed)
tick_name_filtered = filterTickers(tick_names, tick_allowed)

## Define Accounts

In [ ]:
path = "./configs/profiles"
FILE = open(path, 'r')
profiles = FILE.readlines()
FILE.close()
profiles = [profile.strip() for profile in profiles]
profiles

In [ ]:
profiles_names = [
    "Roth IRA",
    "Investment Account",
    "Amazon ",
    "Cash Account",
]
profiles_targets = [
    get_loss(risk_target=0.1),
    None,
    get_loss(),
    None
]

## Get the API Key

In [ ]:
path = "./configs/apikey"
FILE = open(path, 'r')
api_key = FILE.readline()
FILE.close()

## Get the data

In [ ]:
## MAIN BODY 

# Download historical data
start = int(time.mktime(datetime.strptime(start_date, "%d/%m/%Y").timetuple()))
end = int(time.mktime(datetime.strptime(end_date, "%d/%m/%Y").timetuple()))

data_dict = dict()
i = 0

for stock in ticks_filtered:
    if i != 0 and i % 10 == 0:
        print("Sleeping for request limit")
        time.sleep(60)
    
#     querystring = {"to": end, "symbol": stock, "from": start, "resolution": 'D'}
    querystring = f"symbol={stock}&resolution=D&from={start}&to={end}&token={api_key}"

    try:
        response = requests.request("GET", url = (f"https://finnhub.io/api/v1/stock/candle?{querystring}"))
        print(f"https://finnhub.io/api/v1/stock/candle?{querystring}")
        
        data = response.json()
        df = pd.DataFrame.from_dict(data)
        df = df.drop(columns=['s', 'h', 'l', 'o', 'v'])

        # Output time zone: Universal Time Coordinated
        df['time'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in df.t.values]

    except:
        raise ValueError("No data found for " + stock)

    i += 1

    # The download limit is 10 requests per minute
            
    data_dict[stock] = df

data_to_concat = []

In [ ]:
# PRE PROCESS DATA TO A FRIENDLY FORMAT
for key in data_dict:
    data_dict[key] = data_dict[key].rename(columns={"c": key})
    data_to_concat.append(data_dict[key])

price_data = pd.concat(data_to_concat, axis=1, join='inner')
price_data = price_data.loc[:,~price_data.columns.duplicated()].drop(columns='t').set_index('time')

log_ret = np.log(price_data/price_data.shift(1))

cov_mat = log_ret.cov() * 252


## Load Profile Data

In [ ]:
list_of_files = glob.glob('.\profiles\*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file)
df = df[~df.Description.isna()]
df = df[['Account Number', 'Symbol', 'Current Value']]

In [ ]:
profile_makeup = pd.DataFrame(data = np.zeros((len(profiles), len(ticks))), 
                              index=profiles, 
                              columns=ticks)
profile_makeup

In [ ]:
for row_ in df.iterrows():
    if row_[1]["Symbol"] in ticks and  row_[1]["Account Number"] in profiles:
        profile_makeup.loc[row_[1]["Account Number"], row_[1]["Symbol"]] = float(row_[1]["Current Value"][1:])
    elif row_[1]["Account Number"] in profiles:
        profile_makeup.loc[row_[1]["Account Number"], "CASH"] = float(row_[1]["Current Value"][1:])
profile_makeup

## SIMULATE x PORTFOLIOS

In [ ]:
all_wts = np.zeros((num_profiles, len(ticks_filtered)))
port_returns = np.zeros((num_profiles))
port_risk = np.ones((num_profiles))
sharpe_ratio = np.zeros((num_profiles))

for i in range(num_profiles):
    if i % (num_profiles//10)==0:
            print(i, f"{(100 * i / num_profiles)}%")
    
    
    # Portfolio weights
    wts = np.random.uniform(size=len(ticks_filtered))
    
    wts = wts / np.sum(wts)
#     print(w?ts)
    all_wts[i, :] = wts

    # Portfolio Return
    port_ret = np.sum(log_ret.mean() * wts)
    port_ret = (port_ret + 1) ** 252 - 1
    port_returns[i] = port_ret

    # Portfolio Risk
    port_sd = np.sqrt(np.dot(wts.T, np.dot(cov_mat, wts)))
    port_risk[i] = port_sd

    # Portfolio Sharpe Ratio, assuming 0% Risk Free Rate
    sr = port_ret / port_sd
    sharpe_ratio[i] = sr

    # Save portfolios of interest (min var, max return and max SR)
    if sr >= max(sharpe_ratio[0:-1]):
        max_sr_ret = port_ret
        max_sr_risk = port_sd
        max_sr_w = wts
        max_sr = sr

    if port_ret >= max(port_returns[0:-1]):
        max_ret_ret = port_ret
        max_ret_risk = port_sd
        max_ret_w = wts

    if port_sd <= min(port_risk[0:-1]):
        min_var_ret = port_ret
        min_var_risk = port_sd
        min_var_w = wts

In [ ]:
cov_mat = np.array(cov_mat)

In [ ]:
log_ret_mean = np.array(log_ret.mean())

## Get Limits

In [ ]:
# Get Max Sharpe
batch = 100
iterations = 100
LR=0.1

loss_fun = lambda x: -get_sharpe(x)
loss_fun_v = get_loss_v()

start_time = time.time()
best_sharpe_weights, loss = get_gd_weights_adam_batched_exp(len(ticks_filtered), loss_fun_v, batch, iterations, LR, [get_sharpe_v])
print("--- %s seconds ---" % (time.time() - start_time))
loss = loss[0,:,:]
plt.plot(loss.T)
plt.show()
print(get_sharpe(best_sharpe_weights))

start_time = time.time()
best_sharpe_weights, loss = get_gd_weights_adam(len(ticks_filtered), loss_fun, batch, iterations, LR, [get_sharpe])
print("--- %s seconds ---" % (time.time() - start_time))
loss = loss[0,:,:]
plt.plot(loss.T)
plt.show()




In [ ]:
#Get Min Risk
batch = 1000
iterations = 1000
LR=0.01
start_time = time.time()
least_risk_weights, loss = get_gd_weights_adam_batched(len(ticks_filtered), get_risk_v, batch, iterations, LR, [get_risk_v])
print("--- %s seconds ---" % (time.time() - start_time))

loss = loss[0,:,:]

plt.plot(loss.T)
plt.show()

In [ ]:
# Get Max Return
batch = 1000
iterations = 200
LR=0.1

start_time = time.time()
most_return_weights, loss = get_gd_weights_adam_batched(len(ticks_filtered), 
                                           lambda x: npg.negative(get_return_v(x)),
                                           batch, iterations,
                                           LR, [get_return_v])
print("--- %s seconds ---" % (time.time() - start_time))
# start_time = time.time()
# most_return_weights, loss = get_gd_weights_adam(len(ticks_filtered), 
#                                            lambda x: -get_return(x),
#                                            batch, iterations,
#                                            LR, [get_risk])
# print("--- %s seconds ---" % (time.time() - start_time))
loss = loss[0,:,:]

plt.plot(loss.T)
plt.show()

## Get List of Max Returns for Spread of risks

In [ ]:
risk_min = 2.0/100.0
risk_max = 25.0/100.0
risks_count = 24
risks = np.linspace(risk_min, risk_max, risks_count, True)

In [ ]:
b = 100
i = 500
lr = 0.2

In [ ]:
50*np.abs(0.07 - risks)

In [ ]:
batch = [b]*risks_count
iterations = [i]*risks_count
LR=np.array([lr]*risks_count)
# LR = LR*(1.0 + 50*np.abs(0.07 - risks))
# LR[-3]*=10

best_weights_range = np.random.uniform(size=(risks_count, len(ticks_filtered))) 
for r in range(risks_count):
    best_weights_range[r, :], scoring = get_gd_weights_adam(len(ticks_filtered),
                                                  get_loss(risk_target=risks[r]),
                                                  batch[r],
                                                  iterations[r],
                                                  LR[r],
                                                  [get_risk,get_return])
    risk,returns = scoring[0,:,:],scoring[1,:,:]
    fig, ax = plt.subplots(1,2)
    ax[0].plot(risk.T)
    ax[0].set_title('Risk %f' % risks[r])
    ax[1].plot(returns.T)
    ax[1].set_title('Return')
    plt.show()

## Profile to match inflation 

In [ ]:
batch = 20
iterations = 400
LR=1.0

inflation_weights, scoring = get_gd_weights_adam(len(ticks_filtered), 
                                           get_loss(return_target=inflation_rate),
                                           batch, iterations,
                                           LR, [get_risk,get_return])
risk,returns = scoring[0,:,:],scoring[1,:,:]
fig, ax = plt.subplots(1,2)
ax[0].plot(risk.T)
ax[0].set_title('Risk')
ax[1].plot(returns.T)
ax[1].set_title('Return %f'%inflation_rate)
plt.show()

## Build Profiles

In [ ]:
b = 1000
i = 1000
lr = 0.001
batch = [b]*len(profiles)
iterations = [i]*len(profiles)
LR=[lr]*len(profiles)

# batch[2] = 10
# iterations[2] = 200
# LR[2]=0.1
target_weights = []

for i in range(len(profiles)):
    if profiles_targets[i] is not None:
        wts = profile_makeup.loc[profiles[i]].to_numpy()[:-1]
        print(i)
        target_weights_, scoring = get_gd_weights_adam(len(ticks_filtered),
                                                    profiles_targets[i],
                                                    batch[i], iterations[i],
                                                    LR[i], [get_risk,get_return],
                                                    initial = filterTickers(wts, tick_allowed))
        target_weights.append(target_weights_)
        risk,returns = scoring[0,:,:],scoring[1,:,:]
        fig, ax = plt.subplots(1,2)
        ax[0].plot(risk.T)
        ax[0].set_title('Risk')
        ax[1].plot(returns.T)
        ax[1].set_title('Return')
        plt.show()
    else:
        
        wts = profile_makeup.loc[profiles[i]].to_numpy()
        target_weights.append(wts)


In [ ]:
color_list = ["#677737", "#1d27f1", "#e687de", 
              "#764484", "#edac3a", "#e2ca50",
              "#c80727","#7ea0fb", "#4bddac", 
              "#4c6ab5", "#16287c","#e9223b",
              "#db3f9e", "#38f716","#d9510a",
             ]
color_list_accounts = ["#FCBD1F", "#B24157", "#ABC962", 
              "#1DD4AF", "#edac3a", "#e2ca50",
              "#c80727","#7ea0fb", "#4bddac", 
              "#4c6ab5", "#16287c","#e9223b",
              "#db3f9e", "#38f716","#d9510a",
             ]

In [ ]:
## PLOTS
p = figure(
    sizing_mode = "stretch_both", 
#     plot_width=10000,
    title="Efficient frontier. Simulations: " + str(num_profiles),
    tools='box_zoom,wheel_zoom,reset', 
    toolbar_location='above',
)
p.add_tools(CrosshairTool(line_alpha=1, line_color='lightgray', line_width=1))
p.add_tools(HoverTool(tooltips=None))
source = ColumnDataSource(data=dict(risk=port_risk, profit=port_returns))
p.circle(x='risk', y='profit', source=source, line_alpha=0, hover_color='navy', alpha=0.4, hover_alpha=1, size=8)

for i in range(len(ticks_filtered)):
    wts = -0 * np.ones(len(ticks_filtered))
    wts[i]=1.0
    p.circle(get_risk(wts), get_return(wts), color=color_list[i], legend_label=ticks[i], size=8, alpha=0.4)
    
p.circle(get_risk(least_risk_weights), get_return(least_risk_weights), color='tomato', 
         legend_label='Portfolio with min risk', size=12)
p.circle(get_risk(inflation_weights), get_return(inflation_weights), color='limeGreen',
         legend_label='Portfolio matching Inflation', size=12)
p.circle(get_risk(most_return_weights), get_return(most_return_weights), color='firebrick',
         legend_label='Portfolio with max return', size=12)
p.circle(get_risk(best_sharpe_weights), get_return(best_sharpe_weights), color='Green', 
         legend_label='Portfolio with max Sharpe', size=12)

for r in range(risks_count):
    wts = best_weights_range[r,:]
    p.circle(get_risk(wts), get_return(wts), color="teal", legend_label=tick_names[i], size=9)
p.legend.location = "bottom_right"

p.xaxis.axis_label = 'Volatility, or risk (standard deviation)'
p.yaxis.axis_label = 'Annual return'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")

# Portfolio composition. Min variance, max SR, max return
def plot_portfolio_composition(ticks, weights, plot_name):
    print(ticks)
    x = dict()
    c = dict()
    for i in range(len(ticks)):
        if weights[i] >= 1.0:
            x[ticks[i]] = weights[i]
            c[ticks[i]] = color_list[i]
            x[ticks[i]+" "] = weights[i]
            c[ticks[i]+" "] = color_list[i]
        elif weights[i] > 0.0:
            x[ticks[i]] = weights[i]
            c[ticks[i]] = color_list[i]

    plot_data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'stock'})
    plot_data['angle'] = plot_data['value'] / plot_data['value'].sum() * 2 * math.pi
    
    plot_data['color'] = c.values()
    
    print(plot_data)
    print()
    p = figure(width=50, height=50, title=plot_name, toolbar_location=None,sizing_mode = "scale_height",
                      tools="hover", tooltips="@stock: @value{%0.1f}", x_range=(-0.5,0.5))
    p.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                   line_color="white", color='color', source=plot_data)
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    p.outline_line_color = None

    return p

# p_minvar = plot_portfolio_composition(ticks, get_weights(least_risk_weights), "Portfolio with Min Risk")
# p_maxsr = plot_portfolio_composition(ticks, get_weights(inflation_weights), "Portfolio matching Inflation")
# p_maxGD = plot_portfolio_composition(ticks, get_weights(best_sharpe_weights), "Portfolio with max Sharpe GD")
# p_maxret = plot_portfolio_composition(ticks, get_weights(most_return_weights), "Portfolio with max return")


fidelity_pies = []

for i in range(len(profiles)):
    wts = get_weights(profile_makeup.loc[profiles[i]].to_numpy())
    fidelity_pies.append(plot_portfolio_composition(ticks,
                                                    wts,
                                                    profiles_names[i]))
    if wts[-1] == 0.0:
        p.circle(get_risk(wts[:-1]), get_return(wts[:-1]), color=color_list_accounts[i], 
             legend_label=profiles_names[i], size=12)
fidelity_targets = []

for i in range(len(target_weights)):
    fidelity_targets.append(plot_portfolio_composition((ticks if len(target_weights[i]) > len(ticks_filtered) else ticks_filtered),
                                                    get_weights(target_weights[i]),
                                                    profiles_names[i] + " Target"))
    if len(target_weights[i]) == len(ticks_filtered):
        p.circle(get_risk(get_weights(target_weights[i])), get_return(get_weights(target_weights[i])),
                 color=color_list_accounts[i], alpha=0.6,
                 legend_label=profiles_names[i] + " Target", size=12)

# Create dashboard and open new window to show results
layout_ = row([ 
        column([
            p
        ], sizing_mode = "stretch_both"),
        column(fidelity_pies,
               sizing_mode = "stretch_height"), 
        column(fidelity_targets
               , sizing_mode = "stretch_height"),
    ],width = 1500, sizing_mode = "stretch_height")
    
show(layout_)